In [5]:
import numpy as np
import time
#import scipy.interpolate
import torch
#from bspline.BSpline import BSpline
k = 2
t = torch.FloatTensor([0, 1, 2, 3, 4, 5, 6])
c = torch.FloatTensor([-1, 2, 0, -1])
data = torch.FloatTensor([2.5,3])

In [163]:
#Code to test how fast different implementations are
def B(x, k, i, t):
    if k == 0:
       return 1.0 if t[i] <= x < t[i+1] else 0.0
    if t[i+k] == t[i]:
       c1 = 0.0
    else:
       c1 = (x - t[i])/(t[i+k] - t[i]) * B(x, k-1, i, t)
    if t[i+k+1] == t[i+1]:
       c2 = 0.0
    else:
       c2 = (t[i+k+1] - x)/(t[i+k+1] - t[i+1]) * B(x, k-1, i+1, t)
    return c1 + c2

def bspline(x, t, c, k):
    n = len(t) - k - 1
    assert (n >= k+1) and (len(c) >= n)
    pred = x.clone()
    for obs_num in range(x.size(0)):
        pred[obs_num] = sum(c[i] * B(x[obs_num], k, i, t) for i in range(n))

    return pred

#vanilla
start = time.process_time()
prediction_1 = bspline(data, t, c, k)
end = time.process_time()
end-start

0.0057229999999997005

In [164]:
prediction_1

tensor([1.3750, 1.0000])

In [165]:
def bspline_basis(x, t, c, k):
    n = len(t) - k - 1
    #assert (n >= k+1) and (len(c) >= n)
    pred = torch.zeros((x.size(0),n))
    for obs_num in range(x.size(0)):
        for i in range(n):
            pred[obs_num,i] = B(x[obs_num], k, i, t)

    return pred

bspline_basis(data, t, c, k)

tensor([[0.1250, 0.7500, 0.1250, 0.0000],
        [0.0000, 0.5000, 0.5000, 0.0000]])

using fcts of matthias luber

In [116]:
def torch_get_FS(xk):
    """
    Create matrix F required to build the spline base and the penalizing matrix S,
    based on a set of knots xk (ascending order). Pretty much directly from p.201 in Wood (2017)
    :param xk: knots (for now always np.linspace(x.min(), x.max(), n_knots)
    """
    k = len(xk)
    h = torch.diff(xk)
    h_shift_up = h[1:]
    #probably need another one for shift down to work

    D = torch.zeros((k - 2, k))
    D[:,:k-2] += (1 / h[:k - 2])*torch.eye(k-2)
    D[:,1:k-1] += (-1 / h[:k - 2] - 1 / h_shift_up)*torch.eye(k-2)
    D[:,2:k] += (1 / h_shift_up)*torch.eye(k-2)

    B = torch.zeros((k - 2, k - 2))
    B += ((h[:k - 2] + h_shift_up) / 3) * torch.eye(k-2)
    B[:-1,1:] += torch.eye(k-3) * (h_shift_up[:k - 3] / 6)
    B[1:, :-1] += torch.eye(k-3) *(h_shift_up[:k - 3] / 6)
    F_minus = torch.linalg.inv(B) @ D
    F = torch.vstack([torch.zeros(k), F_minus, torch.zeros(k)])
    S = D.T @ torch.linalg.inv(B) @ D
    return F, S

def torch_cr_spl_predict(x, knots, F):
    """
    pretty much the same as cr_spl, this time evaluating it for already given knots and F
    (could probably just be integrated into cr_spl)
    """

    n = len(x)
    k = len(knots)
    base = torch.zeros((n, k))
    #print(n,k,base.shape)
    for i in range(0, len(x)):
        if x[i] < min(knots):
            j = 0
            h = knots[1] - knots[0]
            xik = x[i] - knots[0]
            c_jm = -xik * h / 3
            c_jp = -xik * h / 6
            base[i, :] += c_jm * F[0, :] + c_jp * F[1, :]
            base[i, 0] += 1 - xik / h
            base[i, 1] += xik / h
        elif x[i] > max(knots):
            j = len(knots) - 1
            h = knots[j] - knots[j - 1]
            xik = x[i] - knots[j]
            c_jm = xik * h / 6
            c_jp = xik * h / 3
            base[i, :] += c_jm * F[j - 1, :] + c_jp * F[j, 1]
            base[i, j - 1] += -xik / h
            base[i, j] += 1 + xik / h
        # find interval in which x[i] lies and evaluate accordingly just like in cr_spl
        else:

            j = torch.searchsorted(knots, x[i])
            #print(j)
            x_j = knots[j - 1]
            x_j1 = knots[j]
            h = x_j1 - x_j
            a_jm = (x_j1 - x[i]) / h
            a_jp = (x[i] - x_j) / h
            c_jm = ((x_j1 - x[i]) ** 3 / h - h * (x_j1 - x[i])) / 6
            c_jp = ((x[i] - x_j) ** 3 / h - h * (x[i] - x_j)) / 6
            base[i, :] += c_jm * F[j - 1, :] + c_jp * F[j, :]
            base[i, j - 1] += a_jm
            base[i, j] += a_jp
    return base

In [171]:
F,S = torch_get_FS(t)

In [172]:
torch_cr_spl_predict(data, t, F)

tensor([[ 0.0197, -0.1183,  0.5981,  0.6010, -0.1269,  0.0317, -0.0053],
        [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [166]:
c

tensor([-1.,  2.,  0., -1.])

In [187]:
c2 = torch.tensor([0,-1., 2.,  0.,   -1, 0])

In [191]:
pip install jax

Note: you may need to restart the kernel to use updated packages.


In [192]:
from functools import partial
import jax.numpy as jnp
from jax import jacfwd, jit, lax, vmap, Array


def _check_equidistant_knots(knots: Array) -> bool:
    """
    Checks if knots are equidistants.
    """
    diff = jnp.diff(knots)

    return jnp.allclose(diff, diff[0], 1e-3, 1e-3)


def _check_data_range(x: Array, knots: Array, order: int) -> bool:
    """
    Check that values in x are in the range
    [knots[order - 1], knots[dim(knots) - order]].
    """

    return (
        jnp.min(x) >= knots[order - 1] and jnp.max(x) <= knots[knots.shape[0] - order]
    )


def _check_b_spline_inputs(x: Array, knots: Array, order: int) -> None:
    if not order > 0:
        raise ValueError("Order must be positive")
    if not _check_equidistant_knots(knots):
        raise ValueError("Sorted knots are not equidistant")
    if not _check_data_range(x, knots, order):
        raise ValueError(
            f"Data values are not in the range \
                [{knots[order - 1]}, {knots[knots.shape[0] - order]}]"
        )


@partial(jit, static_argnums=(1, 2))
def create_equidistant_knots(x: Array, order: int, internal_k: int) -> Array:
    """Create the equidistant knots of an (order - 1) B-Spline
    given the data and the number of internal knots.
    dim(knots) = internal_k + order * 2 - 1"""
    min_x = jnp.min(x)
    max_x = jnp.max(x)

    internal_knots = jnp.linspace(min_x, max_x, internal_k)

    step = internal_knots[1] - internal_knots[0]

    left_knots = jnp.linspace(min_x - (step * order), min_x - step, order)
    right_knots = jnp.linspace(max_x + step, max_x + (step * order), order)

    return jnp.concatenate((left_knots, internal_knots, right_knots))


@partial(jit, static_argnums=2)
def _build_basis_vector(x: Array, knots: Array, order: int) -> Array:
    """
    Builds a vector of length (dim(knots) - 1).
    Each entry i is iterativaly updated. At time m,
    the entry i is the evaluation of the basis function
    at the observed value for the m-th order and for the i-th knot.
    The creation of the matrix needs a row-wise (order) loop (f1)
    and a column-wise (knot index) loop (f2).
    """
    k = knots.shape[0] - order
    bv = jnp.full(knots.shape[0] - 1, jnp.nan)

    def basis_per_order(m, bv):
        def basis_per_knot(i, bv):
            def base_case(bv):
                return bv.at[i].set(
                    jnp.where(x >= knots[i], 1.0, 0.0)
                    * jnp.where(x < knots[i + 1], 1.0, 0.0)
                )

            def recursive_case(bv):
                b1 = (x - knots[i]) / (knots[i + m] - knots[i]) * bv[i]
                b2 = (
                    (knots[i + m + 1] - x)
                    / (knots[i + m + 1] - knots[i + 1])
                    * bv[i + 1]
                )

                return bv.at[i].set(b1 + b2)

            return lax.cond(m == 0, base_case, recursive_case, bv)

        return lax.fori_loop(0, k + order - 1, basis_per_knot, bv)

    return lax.fori_loop(0, order, basis_per_order, bv)[:k]


def build_design_matrix_b_spline(x: Array, knots: Array, order: int) -> Array:
    """
    Builds the design matrix for an (order - 1) B-Splines defined by the knots
    at the values in x in an optimized (but not in loco) way.
    Instead of applying the recursive definition of B-splines,
    a matrix of order rows and (dim(knots) - 1) columns for each value in x is created.
    This matrix store the evaluation of the basis function
    at the observed value for the m-th order and for the i-th knot.
    """
    knots = jnp.sort(knots)

    _check_b_spline_inputs(x, knots, order)

    return _build_design_matrix_b_spline_aux(x, knots, order)


@partial(jit, static_argnums=(2))
def _build_design_matrix_b_spline_aux(x: Array, knots: Array, order: int) -> Array:
    """
    Fills the design matrix taking the values in the order-th row and of the first
    (dim(knots) - order) columns from the output of the build_basis_matrix function
    called for each data point.
    """
    return vmap(lambda x: _build_basis_vector(x, knots, order))(x)


def _check_p_spline_matrix_inputs(d: int, r: int) -> None:
    if not d > 0:
        raise ValueError("Matix dimension must be positive")
    if not r >= 0:
        raise ValueError("Difference order must be non-negative")


def build_p_spline_matrix(d: int, r: int = 1) -> Array:
    _check_p_spline_matrix_inputs(d, r)

    return _build_p_spline_matrix_aux(d, r)


@partial(jit, static_argnums=(0, 1))
def _build_p_spline_matrix_aux(d: int, r: int = 1) -> Array:
    """
    Builds (d x d) a penalty matrix with differences of order r.
    """
    D = jnp.diff(jnp.identity(d), r, axis=0)

    return D.T @ D

In [195]:
data.numpy()

array([2.5, 3. ], dtype=float32)

In [233]:
x = jnp.arange(0, 200)
internal_k = 15
order = 2

knots = create_equidistant_knots(x, order, internal_k)

X = build_design_matrix_b_spline(x, knots, order)
beta = np.random.randn(X.shape[1])

#scipy_spl = BSpline(knots, beta, order - 1)

#assert np.allclose(X @ beta, scipy_spl(x), 1e-3, 1e-3)
X.shape

(200, 17)

In [239]:
X = build_design_matrix_b_spline(x, knots, order)

In [227]:
from scipy.interpolate import BSpline, splev

In [230]:
#scipy_spl = BSpline(knots, beta, order - 1)
#scipy_spl(x)
splev(x, [knots, torch.zeros(12), 3], der=0)

array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0

In [188]:
basis = torch_cr_spl_predict(data, t, F)
torch.matmul(basis,c2)

RuntimeError: size mismatch, got 2, 2x7,6

In [47]:
#scipy
start = time.time()
spl = scipy.interpolate.BSpline(t, c, k)
spl(data)
end = time.time()
end-start

0.00033092498779296875

In [48]:
#pytorch based on scipy
start = time.process_time()
bspline = BSpline(start=0, end=10000, n_bases=4, spline_order=2)
mat = bspline.predict(torch.tensor(data))
prediction_2 = torch.matmul(mat,torch.tensor(c,dtype=torch.float32))
end = time.process_time()
end-start

0.012907999999999475

In [6]:
def deBoor(k: int, x: int, t, c, p: int):
    """Evaluates S(x).

    Arguments
    ---------
    k: Index of knot interval that contains x.
    x: Position.
    t: Array of knot positions, needs to be padded as described above.
    c: Array of control points.
    p: Degree of B-spline.
    """
    d = [c[j + k - p] for j in range(0, p + 1)]

    for r in range(1, p + 1):
        for j in range(p, r - 1, -1):
            alpha = (x - t[j + k - p]) / (t[j + 1 + k - r] - t[j + k - p])
            d[j] = (1.0 - alpha) * d[j - 1] + alpha * d[j]

    return d[p]

In [63]:
data = np.array(np.linspace((0,6),0.1))

In [64]:
data

array([[0.00000000e+00, 6.00000000e+00],
       [2.04081633e-03, 5.87959184e+00],
       [4.08163265e-03, 5.75918367e+00],
       [6.12244898e-03, 5.63877551e+00],
       [8.16326531e-03, 5.51836735e+00],
       [1.02040816e-02, 5.39795918e+00],
       [1.22448980e-02, 5.27755102e+00],
       [1.42857143e-02, 5.15714286e+00],
       [1.63265306e-02, 5.03673469e+00],
       [1.83673469e-02, 4.91632653e+00],
       [2.04081633e-02, 4.79591837e+00],
       [2.24489796e-02, 4.67551020e+00],
       [2.44897959e-02, 4.55510204e+00],
       [2.65306122e-02, 4.43469388e+00],
       [2.85714286e-02, 4.31428571e+00],
       [3.06122449e-02, 4.19387755e+00],
       [3.26530612e-02, 4.07346939e+00],
       [3.46938776e-02, 3.95306122e+00],
       [3.67346939e-02, 3.83265306e+00],
       [3.87755102e-02, 3.71224490e+00],
       [4.08163265e-02, 3.59183673e+00],
       [4.28571429e-02, 3.47142857e+00],
       [4.48979592e-02, 3.35102041e+00],
       [4.69387755e-02, 3.23061224e+00],
       [4.897959

In [3]:
#t = torch.tensor(t,dtype=torch.float32)
#c = torch.tensor(c,dtype=torch.float32)
import functorch

In [75]:
class deBoor():
    def __init__(self,t,c,p,x):
        self.t=t
        self.c=c
        self.p=p
        self.n = t.size(0) - 2 * self.p


    def compute(self, x, k):
        """Evaluates S(x).

        Arguments
        ---------
        k: Index of knot interval that contains x.
        x: Position.
        t: Array of knot positions, needs to be padded as described above.
        c: Array of control points.
        p: Degree of B-spline.
        """

        d = [self.c[j + k - self.p] for j in range(0, self.p + 1)]

        for r in range(1, self.p + 1):
            for j in range(self.p, r - 1, -1):
                alpha = (x - self.t[j + k - self.p]) / (self.t[j + 1 + k - r] - self.t[j + k - self.p])
                d[j] = (1.0 - alpha) * d[j - 1] + alpha * d[j]

        return d[self.p]


start = time.process_time()

#k = torch.searchsorted(self.t, x) - 1
n = t.size(0) - 2 * 2
k = torch.searchsorted(t, data) - 1
#k[k > (n - 1)] = 2 + 1
#k[k > (n - 1)] = 2 + (n - 1) - 1

deBoor_obj = deBoor(t=t,c=c,p=torch.tensor([2]),x=data)
deBorr_vec = functorch.vmap(deBoor_obj.compute)

prediction_3 = deBorr_vec(torch.unsqueeze(data,0), torch.unsqueeze(k,0))
end = time.process_time()
end-start

0.0034320000000001016

In [76]:
prediction_3

tensor([[1.3750, 1.0000]])

In [68]:
data2 = torch.unsqueeze(data,0)
k2 = torch.unsqueeze(k,0)
input_dk = torch.cat((data2,k2))
input_dk

tensor([[2.5000, 3.0000],
        [2.0000, 2.0000]])

In [243]:
pip install functorch

  Using cached torch-1.13.1-cp37-none-macosx_10_9_x86_64.whl (135.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1
    Uninstalling torch-1.10.1:
      Successfully uninstalled torch-1.10.1
Note: you may need to restart the kernel to use updated packages.


In [51]:
print(prediction_1)

[1.375]


In [52]:
print(prediction_2)

tensor([0.5075])


In [55]:
print(prediction_3)

[1.375]
